After downloading the data, with bzcat Downloads/RC_2015-01.bz2 | head | less is possible to see the structure of the json at the moment:

In [4]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url= "https://doc.gold.ac.uk:65000/user/acost051/files/BDA/reddit_data_json.jpg")

It would be necessary to download, and let ready the Hive-JSON-Serde tools for working.
After that, in hive, a new database would be created (for this time, was called 05br).
Then, you add to hive the Hive-JSON-Serde tool.
The next step is creating a table, that would be call "rc", and fill it with the data.

#Creation of the table
CREATE TABLE rc ( subreddit STRING, author STRING, name STRING, id STRING, parent_id STRING )
  ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe' STORED AS TEXTFILE;
  
#Fill the table with data
LOAD DATA LOCAL INPATH 'Downloads/RC_2015-01.bz2' OVERWRITE INTO TABLE rc

With the table full of data, now a query would be done.
This query will be save in the directory "home/acost051/rcreply", showing how many times a user answers to another user.

INSERT OVERWRITE LOCAL DIRECTORY 'home/acost051/rcreply' ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' 
    > SELECT a.author, b.author, COUNT(*) FROM rc a JOIN rc b ON a.name = b.parent_id GROUP BY a.author, b.author;
    
For obtain an output:
cat BigData/Reddit/home/acost051/rcreply/* | grep -v '\W1$' | grep -v '^\[deleted\][[:space:]]' | grep -v '[[:space:]]\[deleted\][[:space:]]' > rcreply43.ncol

(The name "rcreply43.ncol" was used to make a difference between other files that did not work as expected)

In [5]:
#Example of the first 33 rows that is possible to find in the field "rcreply43.ncol"
Image(url="https://doc.gold.ac.uk:65000/user/acost051/files/BDA/reddit_1.jpg")

With that file, now would be created some plots showing the distribution.

In [ ]:
import matplotlib.pyplot as p
import igraph

G = igraph.Graph.Read_Ncol('C:/MSC Data Science/Big Data Applications/reddit/rcreply43.ncol')

hi = G.degree_distribution(mode='IN')
ho = G.degree_distribution(mode='OUT')

#SOME CHANGES HAVE TO BE DONE BECAUSE OF THE DIFFERENT VERSIONS OF PYTHON
p.plot([x[0] for x in hi.bins()], [x[2] for x in hi.bins()])
p.show()
p.plot([x[0] for x in ho.bins()], [x[2] for x in ho.bins()])
p.show()

In [6]:
#Graph of the hi
Image(url="https://doc.gold.ac.uk:65000/user/acost051/files/BDA/rcreply_hi.png")

In [7]:
#Graph of the ho
Image(url="https://doc.gold.ac.uk:65000/user/acost051/files/BDA/rcreply_ho.png")

In [8]:
Image(url="https://doc.gold.ac.uk:65000/user/acost051/files/BDA/rcbigdata.jpg")

Testing the multiple outputs at once.
Query:

insert overwrite local directory '/home/acost051/rcall'
row format delimited fields terminated by '\t'
select a.subreddit, a.author, b.author, count(*) from
rc a join rc b on a.name = b.parent_id
group by a.subreddit, a.author, b.author;

Output
cat BigData/Reddit/home/acost051/rcall/* | grep -v '\W1$' | grep -v '^\[deleted\][[:space:]]' | grep -v '[[:space:]]\[deleted\][[:space:]]' > rcall.ncol

Result:

In [9]:
Image(url="https://doc.gold.ac.uk:65000/user/acost051/files/BDA/rcall.jpg")

The output was obtained to see how it looks.
The real next step was creating a new table (called rcall) in the database.

create table rcall (author1 STRING, author2 STRING, count INT) partitioned by (subreddit STRING)
row format delimited fields terminated by '\t';

Next step was setting the Hive parameters:

set hive.exec.dynamic.partition.mode = nonstrict;
set hive.exec.max.dynamic.partitions = 50000;
set hive.exec.max.dynamic.partitions.pernode = 50000;

There are two options for obtain multiple outputs at once.
The first query:
select c.* from (select subreddit, count(*) as count from rc group by subreddit) c where c.count > 10000;

The second query:
select subreddit, count(*) from rc group by subreddit having count(*) > 10000;

The first one is done in 299.226 seconds, while the second one is done in 263.433 seconds. The difference between them, is that the first query is using a subquery, and that is probably why needs more time to be complete.

Now, the hive settings would change to 1000 for each one:

set hive.exec.max.dynamic.partitions = 1000;
set hive.exec.max.dynamic.partitions.pernode = 1000;
set hive.optimize.sort.dynamic.partition = true;

The final query that generated a big number of individually labelled directories (reason why it needs 924.586 seconds to be complete, doing 7 jobs):

insert overwrite table rcall partition (subreddit)
select a.author, b.author, count(*), a.subreddit from 
rc a join rc b on a.name = b.parent_id 
  join (select subreddit, count(*) as count from rc group by subreddit) c on a.subreddit = c.subreddit
where c.count > 10000
group by a.author, b.author, a.subreddit;

Was not possible to obtain the output with the hadoop commands, but the results that were show in the screen are:

In [10]:
Image(url="https://doc.gold.ac.uk:65000/user/acost051/files/BDA/partition_rcall.jpg")

With the file "rcbigada_learn.ncol", that was obtained with this query:

insert overwrite local directory '/home/acost051/rcbigdata'
row format delimited fields terminated by '\t'
select a.author, b.author, count(*) from rc a join rc b on a.name = b.parent_id
where a.subreddit = 'learnprogramming'
group by a.author, b.author;

And the output:
cat BigData/Reddit/home/acost051/rcbigdata/* | grep -v '\W1$' | grep -v '^\[deleted\][[:space:]]' | grep -v '[[:space:]]\[deleted\][[:space:]]' > rcbigdata_learn.ncol


In [ ]:
import igraph

# parse the graph
G = igraph.Graph.Read_Ncol("rcbigdata_learn.ncol") #THIS IS GIVING AN ERROR, THAT WASN´T POSSIBLE TO SOLVE WHAT WAS IT

# delete the vertex corresponding to the [deleted] pseudo-user
G.delete_vertices(G.vs['name'].index('[deleted]'))

# compute the geometric mean of the in- and out-degree for each vertex
mdegree = map(lambda x,y: sqrt(x*y), G.indegree(), G.outdegree())

i = mdegree.index(max(mdegree))

print ("most connected commenter in '%s': %s (indegree %d, outdegree %d)" % (subreddit, G.vs[i]['name'], G.indegree(i), G.outdegree(i) ))

There where different problems that did not let obtain the graphs.